In [ ]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd

# Create an Elasticsearch client
es = Elasticsearch(hosts=["http://localhost:9200"])  # Replace "localhost" with your Elasticsearch server's address
es


<Elasticsearch(['http://localhost:9200'])>

In [90]:
# Read CSV data using pandas
csv_file = "C:/Users/zohreh/Desktop/books_info.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_file)

In [91]:
# Convert the DataFrame to a list of dictionaries
data = df.to_dict(orient="records")

In [ ]:
pip install requests

In [95]:
import requests

endpoint = 'http://localhost:9200'
index_name = 'book_index'
index_mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "url": {"type": "keyword"},
            "contributors": {"type": "keyword"},
            "date": {"type": "keyword"},
            "format": {"type": "keyword"},
            "full_text_url": {"type": "keyword"},
            "trove_id": {"type": "keyword"},
            "language": {"type": "keyword"},
            "rights": {"type": "keyword"},
            "pages": {"type": "integer"},
            "form": {"type": "keyword"},
            "volume": {"type": "keyword"},
            "children": {"type": "keyword"},
            "parent": {"type": "keyword"},
            "text_downloaded": {"type": "keyword"},
            "text_file": {"type": "keyword"}
        }
    }
}

response = requests.put(f'{endpoint}/{index_name}', json=index_mapping)
if response.status_code == 200:
    print("Index created successfully!")
else:
    print("Failed to create the index.")

Failed to create the index.


In [ ]:
from elasticsearch import Elasticsearch
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

es = Elasticsearch(hosts=['http://localhost:9200'])
print("Elasticsearch is ready.")
cluster = Cluster(['localhost'])
session = cluster.connect()
print("Cassandra is ready.\n")

#ElasticSearch settings:
index_name = 'book_index'
batch_size = 1000
sort_field = '_doc'
sort_order = 'asc'
i=0
search_after = None

#Cassandra Settings:
insert_query_for_parents = "INSERT INTO book2.parents (trove_id,title,url,contributors,date,format,pages) VALUES (?,?,?,?,?,?,?)"
create_table_query_parents= "CREATE TABLE book2.parents (\
    title text,\
    url text,\
    contributors text,\
    date text,\
    format text,\
    trove_id text PRIMARY KEY,\
    pages int\
);"
# Prepare the insert statement
insert_statement = session.prepare(insert_query_for_parents)

#STEP 1: Find parents and add them to goly.parents table:
while True:
    res = es.search(
        index=index_name,
        body={
            'query': {
        "bool": {
            "must": [
                {
                    "exists": {
                        "field": "children"
                    }
                },
                {
                    "bool": {
                        "must_not": [
                            {
                                "term": {
                                    "children": ""
                                }
                            }
                        ]
                    }
                }
            ]
        }
    },
            'sort': [{sort_field: sort_order}],
            'size': batch_size,
            'search_after': search_after
        }
    )

    if res['hits']['hits']:
        last_hit = res['hits']['hits'][-1]
        search_after = last_hit['sort']
        
        for hit in res['hits']['hits']:
            doc_id = hit['_id']
            doc = hit['_source']
            i=i+1
            # Perform any necessary data transformations or mappings here
            row = (doc['trove_id'], doc['title'], doc['url'], doc['contributors'], doc['date'], doc['format'], int(doc['pages']))
            session.execute(insert_statement, row)
            print(f"{i} docs added to book2.parents.", end="\r")
    else:
        break

print(f"\nTotal parents documents:{i}\n")

In [105]:
#step1
import csv
from elasticsearch import Elasticsearch

def counter():
    i = 0
    def increment():
        nonlocal i
        i += 1
        return i
    return increment

file=open(csv_file, 'r', encoding='utf-8')
i = counter()

reader = csv.DictReader(file)
my_body = []
for row in reader:
    index_name = f"book_index_{i()}"
    my_body.append({
        "index": {
            "_index": 'book_index',
            "_id": index_name
        }
    })
    my_body.append(dict(row))

print(len(my_body))
print(my_body[:5])



# Call es.bulk() method to send data to Elasticsearch
response = es.bulk(body=my_body)
#csv_file.close()  # Close the CSV file after use



84348
[{'index': {'_index': 'book_index', '_id': 'book_index_1'}}, {'title': 'Goliath Joe, fisherman / by Charles Thackeray (Wobbegong)', 'url': 'https://trove.nla.gov.au/work/10013347', 'contributors': 'Thackeray, Charles', 'date': '1900-1919', 'format': 'Book|Book/Illustrated', 'fulltext_url': 'https://nla.gov.au/nla.obj-2831231419', 'trove_id': 'nla.obj-2831231419', 'language': 'English', 'rights': 'Out of Copyright|http://rightsstatements.org/vocab/NKC/1.0/', 'pages': '130', 'form': 'Book', 'volume': '', 'parent': '', 'children': '', 'text_downloaded': 'True', 'text_file': 'goliath-joe-fisherman-by-charles-thackeray-wob-nla.obj-2831231419.txt'}, {'index': {'_index': 'book_index', '_id': 'book_index_2'}}, {'title': 'Grammar of the Narrinyeri tribe of Australian Aborigines / by the late Rev. G. Taplin', 'url': 'https://trove.nla.gov.au/work/10029401', 'contributors': 'Taplin, George', 'date': '1878-1880', 'format': 'Book|Government publication', 'fulltext_url': 'http://nla.gov.au/nla

C:\Users\zohreh\AppData\Local\Temp\ipykernel_26924\121720579.py:34: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'operations' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.bulk(body=my_body)


In [ ]:
#children
es = Elasticsearch(['http://localhost:9200'])  # Replace 'localhost' with your Elasticsearch cluster address

# Connect to Cassandra
cluster = Cluster(['localhost'])  # Replace 'localhost' with your Cassandra cluster address
session = cluster.connect()

# Define the keyspace and table
session.execute("CREATE KEYSPACE IF NOT EXISTS my_book WITH replication = {'class':'SimpleStrategy', 'replication_factor':1}")
session.set_keyspace('my_book')
children=session.execute("CREATE TABLE IF NOT EXISTS children_books (trove_id TEXT PRIMARY KEY, title TEXT, url TEXT, contributors TEXT, date TEXT, format TEXT, pages INT)")
parents=session.execute("CREATE TABLE IF NOT EXISTS parents_books (trove_id TEXT PRIMARY KEY, title TEXT, url TEXT, contributors TEXT, date TEXT, format TEXT, pages INT)")

In [106]:


# Retrieve data from Elasticsearch
es_index = 'book_index'  # Replace 'book_index' with the actual Elasticsearch index name
sort_field = '_doc'  # Replace 'your_sort_field' with the actual field to sort on
sort_order = 'asc'  # Replace 'asc' with 'desc' if you want to sort in descending order
batch_size = 1000  # Adjust the batch size as per your requirement
search_after = None
i = 0

# Prepare insert statement for children_books table
insert = session.prepare("INSERT INTO children_books (trove_id, title, url, contributors, date, format, pages) VALUES (?, ?, ?, ?, ?, ?, ?)")

# Retrieve and store data from Elasticsearch
while True:
    result = es.search(
        index=es_index,
        body={
            'query': {
                "bool": {
                    "must": [
                        {
                            "exists": {
                                "field": "parent"
                            }
                        },
                        {
                            "regexp": {
                                "parent": ".+"
                            }
                        }
                    ]
                }
            },
            'sort': [{sort_field: sort_order}],
            'size': batch_size,
            'search_after': search_after
        }
    )


    if result['hits']['hits']:
        last_hit = result['hits']['hits'][-1]
        search_after = last_hit['sort']

        for hit in result['hits']['hits']:
            doc_id = hit['_id']
            doc = hit['_source']
            i += 1
            # Perform any necessary data transformations or mappings here
            date= doc['date']
            if len(date) <= 4:
                row = (doc['trove_id'], doc['title'], doc['url'], doc['contributors'], date, doc['format'], int(doc['pages']))
            else:
                row = (doc['trove_id'], doc['title'], doc['url'], doc['contributors'], date[5:], doc['format'], int(doc['pages']))
            session.execute(insert, row)
            
    else:
        break

print(f"\nTotal children documents: {i}\n")

# Close the connections
session.shutdown()
cluster.shutdown()


C:\Users\zohreh\AppData\Local\Temp\ipykernel_26924\344606885.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(



Total children documents: 1423



In [ ]:
#parents
from elasticsearch import Elasticsearch
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

es = Elasticsearch(hosts=['http://localhost:9200'])
cluster = Cluster(['localhost'])
session = cluster.connect()
# Create the keyspace if not exists
session.execute("CREATE KEYSPACE IF NOT EXISTS my_book WITH replication = {'class':'SimpleStrategy', 'replication_factor':1}")
session.set_keyspace('my_book')
# Elasticsearch settings
index_name = 'my_index3'
batch_size = 1000
sort_field = '_doc'
sort_order = 'asc'
i = 0
search_after = None


In [ ]:

# Cassandra settings
insert_query = "INSERT INTO my_book.parents (trove_id, title, url, contributors, date, format, pages) VALUES (?, ?, ?, ?, ?, ?, ?)"
table_query_parents = "CREATE TABLE IF NOT EXISTS my_book.parents (title text, url text, contributors text, date text, format text, trove_id text PRIMARY KEY, pages int);"

# Create the table in Cassandra
session.execute(table_query_parents)

# Prepare the insert statement
insert_statement = session.prepare(insert_query)

while True:
    result = es.search(
        index='book_index',
        body={
            'query': {
                "bool": {
                    "must": [
                        {
                            "exists": {
                                "field": "children"
                            }
                        },
                        {
                            "bool": {
                                "must_not": [
                                    {
                                        "term": {
                                            "children": ""
                                        }
                                    }
                                ]
                            }
                        }
                    ]
                }
            },
            'sort': [{sort_field: sort_order}],
            'size': batch_size,
            'search_after': search_after
        }
    )

    if result['hits']['hits']:
        last_hit = result['hits']['hits'][-1]
        search_after = last_hit['sort']
        
        for hit in result['hits']['hits']:
            doc_id = hit['_id']
            doc = hit['_source']
            i=i+1
            # Perform any necessary data transformations or mappings here
            row = (doc['trove_id'], doc['title'], doc['url'], doc['contributors'], doc['date'], doc['format'], int(doc['pages']))
            session.execute(insert_statement, row)
    else:
        break

print(f"\nTotal parents documents:{i}\n")

# Close the connections
session.shutdown()
cluster.shutdown()
